# **Capstone Project: London Transport Policy Analysis**
## **Models for Percent Change Dataset**
## Andrea Broaddus

### **This notebook has three sections:**

**1**.   Baseline Logistic Regression Model and Feature Selection

**2**.   Comparison of Random Forest Classifiers Correcting for Imbalanced Dataset

*     Model 1. Random Forest Baseline, no correction for imbalanced classes
*   Model 2. Random Forest with Oversampling
*   Model 3. Random Forest with Class Weights

  
**3**.   Neural Network Models

*   Testing Optimization Algorithms
*   Hyperparameter Tuning

In [ ]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile, chi2, f_regression
from sklearn.compose import make_column_transformer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=Warning)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load the 2011 feature dataset
Xpc = pd.read_csv('/content/drive/MyDrive/MLAI_Haas/Capstone/Xpc.csv')
Xpc.info()

In [ ]:
#Load the target dataset
y = pd.read_csv('/content/drive/MyDrive/MLAI_Haas/Capstone/y.csv')
y.value_counts()

In [ ]:
#Split the data into train and test sets
Xpc_train, Xpc_test, y_train, y_test = train_test_split(Xpc, y, test_size=0.3, random_state = 42)
print(Xpc_train.shape)
print(y_train.shape)
print(Xpc_test.shape)
print(y_test.shape)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

# **1. Baseline Logistic Regression Model**

In [ ]:
#Estimate a baseline Logistic Regression model
random_seed = 42
lr01=LogisticRegression(class_weight='balanced').fit(Xpc_train, y_train)
lr01_train_preds = lr01.predict(Xpc_train)
lr01_test_preds = lr01.predict(Xpc_test)
lr01_report = classification_report(y_test, lr01_test_preds)
print(lr01_report)
ConfusionMatrixDisplay.from_predictions(y_test, lr01_test_preds)

In [ ]:
# Access the coefficients from the fitted estimator
coefficients_01 = lr01.coef_[0]

# Get the feature names
feature_names_01 = Xpc_train.columns

#Plot coefficients in order of importance
feature_importance_01 = pd.DataFrame({'Feature': feature_names_01, 'Importance': np.float64(coefficients_01)})
feature_importance_01 = feature_importance_01.sort_values('Importance', ascending=True)
feature_importance_01.plot(x='Feature', y='Importance', kind='barh', figsize=(6,6))

## **Feature Selection**


In [ ]:
#Use Select from Model tool with logistic regression to identify most valuable features
from sklearn.feature_selection import SelectFromModel
random_seed = 42
lr01_select = SelectFromModel(LogisticRegression(C = .1, penalty = 'l1', solver = 'liblinear', class_weight='balanced', random_state = 42))
lr01_select.fit_transform(Xpc_train, y_train)
lr01_select_model = SelectFromModel(lr01_select, prefit=True)

In [ ]:
# Access the coefficients from the fitted estimator
coefficients = lr01_select.estimator_.coef_[0]

# Assuming X_train is a pandas DataFrame with column names
# Get the feature names from X_train.columns
feature_names = Xpc_train.columns

# Use get_support to get a boolean mask of the selected features
support = lr01_select.get_support()

# Get the selected feature names
selected_features = feature_names[support]

# Get the coefficients corresponding to the selected features
selected_coefficients = coefficients[support]

#Plot coefficients in order of importance
feature_importance = pd.DataFrame({'Feature': selected_features, 'Importance': np.float64(selected_coefficients)})
feature_importance = feature_importance.sort_values('Importance', ascending=True)
feature_importance.plot(x='Feature', y='Importance', kind='barh', figsize=(10, 6))

# **2. Comparison of Classifiers Correcting for Imbalanced Dataset**

This is a highly imbalanced dataset, with the target (positive) case only 2%. We will test three methods of correcting for imbalance and improving model preditive power.

Two methods of correcting for an imbalanced target class were compared to a baseline Random Forest Classifier model using default settings. Each model was scored using stratified k-fold validation, and GridSearchCV was used for hyperparameter tuning.


*   Model 1. Random Forest Baseline, no correction for imbalanced classes
*   Model 2. Random Forest with Oversampling
*   Model 3. Random Forest with Class Weights

•	Balancing class weights. The model parameter class_weight is set to “balanced,” meaning the classes will be weighted in the loss function inversely proportional to their frequency in the dataset. The estimator works to minimize the error on the more heavily weighted positive target class.

•	Oversampling from the target class. The target feature is sampled to create a “balanced” dataset with 50% positive and negative cases, prior to running the estimator. In this method the negative cases are undersampled.


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import roc_auc_score, RocCurveDisplay, PrecisionRecallDisplay, DetCurveDisplay

Recall: The ability of a model to find all the relevant cases within a data set. The number of true positives divided by the number of true positives plus the number of false negatives.

In most high-risk detection cases (like cancer), recall is a more important evaluation metric than precision.

In the case of credit card fraud detection, we want to avoid false negatives as much as possible. Fraud transactions cost us a lot and thus we want to take appropriate measures to prevent them. A false negative case means that a fraud-positive transaction is assessed to genuine transaction, which is detrimental. In this use case, false positives (a genuine transaction as fraud-positive) are not as important as preventing a fraud.

AUC (Area Under The Curve) ROC (Receiver Operating Characteristics) curve.

ROC is a probability curve and AUC represents the degree or measure of separability. It tells how much the model is capable of distinguishing between classes. Higher the AUC, the better the model is at predicting 0 classes as 0 and 1 classes as 1.

### **Model 1. Random Forest Baseline, no correction for imbalanced classes**

In [ ]:
#Baseline model with Random Forest classifier and no correction for class weights

random_seed = 42
kf = StratifiedKFold(n_splits=5, shuffle=False)
rf = RandomForestClassifier(n_estimators=100, random_state=13)
score = cross_val_score(rf, Xpc_train, np.ravel(y_train,order='C'), cv=kf, scoring='recall', verbose=0)

print("Training Cross Validation Recall scores are: {}".format(score))
print("Training Average Cross Validation Recall score: {}".format(score.mean()))

In [ ]:
#Hyperparameter Tuning Using GridSearchCV

params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]}

grid_rf = GridSearchCV(rf, param_grid=params, cv=kf,
                          scoring='recall').fit(Xpc_train,np.ravel(y_train,order='C'))

print('Best parameters:', grid_rf.best_params_)
print('Best score:', grid_rf.best_score_)

In [ ]:
#Evaluation Metrics
random_seed = 42
y_pred = grid_rf.predict(Xpc_test)

rf_Recall = recall_score(y_test, y_pred)
rf_Precision = precision_score(y_test, y_pred)
rf_f1 = f1_score(y_test, y_pred)
rf_accuracy = accuracy_score(y_test, y_pred)

ROCAUCscore = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC Curve for Random Forest with No Under/Oversampling: {ROCAUCscore:.4f}")

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
#Summary table

ndf = [(rf_Recall, rf_Precision, rf_f1, rf_accuracy)]

rf_score = pd.DataFrame(data = ndf, columns=['Recall','Precision','F1 Score', 'Accuracy'])
rf_score.insert(0, 'Random Forest with', 'No Under/Oversampling')
rf_score

### **Model 2. Random Forest with Oversampling**

In [ ]:
#Random Oversampling using imbalanced learning pipeline
from imblearn.over_sampling import RandomOverSampler

#Define oversampling strategy
ros = RandomOverSampler(random_state=42)

# fit and apply the transform
X_over, y_over = ros.fit_resample(Xpc_train, y_train)

# Convert y_over to a Pandas Series to use value_counts()
#y_over_series = pd.Series(y_over)

# Use y_over_series for value_counts()
print('Genuine:', y_over.value_counts()[0], '/', round(y_over.value_counts()[0]/len(y_over) * 100,2), '% of the dataset')
print('Frauds:', y_over.value_counts()[1], '/',round(y_over.value_counts()[1]/len(y_over) * 100,2), '% of the dataset')

In [ ]:
#Imbalanced Learning Pipeline
from imblearn.pipeline import Pipeline, make_pipeline

#Define and train the model
random_seed = 42
kf = StratifiedKFold(n_splits=5, shuffle=False)
random_overs_pipeline = make_pipeline(RandomOverSampler(random_state=42),
                              RandomForestClassifier(n_estimators=100, random_state=13))

#Score the model
score2 = cross_val_score(random_overs_pipeline, Xpc_train, y_train, scoring='recall', cv=kf)

print("Training Cross Validation Recall Scores are: {}".format(score2))
print("Training Average Cross Validation Recall score: {}".format(score2.mean()))

In [ ]:
#Use GridSearch to tune the hyperparameters
random_seed = 42
new_params = {'randomforestclassifier__' + key: params[key] for key in params}
grid_over_rf = GridSearchCV(random_overs_pipeline, param_grid=new_params, cv=kf, scoring='recall',
                        return_train_score=True)
grid_over_rf.fit(Xpc_train, y_train)

print('Best parameters:', grid_over_rf.best_params_)
print('Best score:', grid_over_rf.best_score_)

In [ ]:
#Evaluation Metrics
random_seed = 42
y_pred = grid_over_rf.best_estimator_.named_steps['randomforestclassifier'].predict(Xpc_test)

over_rf_Recall = recall_score(y_test, y_pred)
over_rf_Precision = precision_score(y_test, y_pred)
over_rf_f1 = f1_score(y_test, y_pred)
over_rf_accuracy = accuracy_score(y_test, y_pred)

ROCAUCscore = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC Curve for Random Forest with Random Oversampling: {ROCAUCscore:.4f}")

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
#Summary table
ndf = [(over_rf_Recall, over_rf_Precision, over_rf_f1, over_rf_accuracy)]

over_rf_score = pd.DataFrame(data = ndf, columns=['Recall','Precision','F1 Score', 'Accuracy'])
over_rf_score.insert(0, 'Random Forest with', 'Random Oversampling')
over_rf_score

### **Model 3. Random Forest with Class Weights**

In [ ]:
#Define Random Forest classifier using class weight parameter
random_seed = 42
rfb = RandomForestClassifier(n_estimators=100, random_state=13, class_weight="balanced")

#Score the model
score5 = cross_val_score(rfb, Xpc_train, y_train, cv=kf, scoring='recall')

print("Training Cross Validation Recall scores are: {}".format(score5))
print("Training Average Cross Validation Recall score: {}".format(score5.mean()))

In [ ]:
#Use GridSearch to tune the hyperparameters
random_seed = 42
grid_rfb = GridSearchCV(rfb, param_grid=params, cv=kf,
                          scoring='recall').fit(Xpc_train, y_train)

print('Best parameters:', grid_rfb.best_params_)
print('Best score:', grid_rfb.best_score_)

In [ ]:
#Evaluation metrics
random_seed = 42
y_pred = grid_rfb.predict(Xpc_test)

grid_rfb_Recall = recall_score(y_test, y_pred)
grid_rfb_Precision = precision_score(y_test, y_pred)
grid_rfb_f1 = f1_score(y_test, y_pred)
grid_rfb_accuracy = accuracy_score(y_test, y_pred)

ROCAUCscore = roc_auc_score(y_test, y_pred)
print(f"AUC-ROC Curve for Random Forest with Class Weights: {ROCAUCscore:.4f}")

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
#Summary table
ndf = [(grid_rfb_Recall, grid_rfb_Precision, grid_rfb_f1, grid_rfb_accuracy)]

grid_rfb_score = pd.DataFrame(data = ndf, columns=['Recall','Precision','F1 Score', 'Accuracy'])
grid_rfb_score.insert(0, 'Random Forest with', 'Class weights')
grid_rfb_score

In [ ]:
#Final summary table
predictions = pd.concat([rf_score, over_rf_score, grid_rfb_score], ignore_index=True, sort=False)
predictions.sort_values(by=['Recall'], ascending=False)

## **Visulazation of Results**

In [ ]:
#Plot of ROC and Precision-Recall curves
fig, axes = plt.subplots(1, 2, figsize=(10, 10))
ax_roc, ax_pr = axes[0], axes[1]

classifiers = {
    "Random Forest": grid_rf,
    "Random Forest with Over Sampling": grid_over_rf,
    "Random Forest with Class weights": grid_rfb}

for name, clf in classifiers.items():
    clf.fit(Xpc_train, y_train)
    RocCurveDisplay.from_estimator(clf, Xpc_test, y_test, ax=ax_roc, name=name)
    PrecisionRecallDisplay.from_estimator(clf, Xpc_test, y_test, ax=ax_pr, name=name)

ax_roc.set_title("Receiver Operating Characteristic (ROC) curves")
ax_pr.set_title("Precision Recall curves")
plt.legend()
plt.show()

In [ ]:
#Plot of Detection-Error curve
fig, axes = plt.subplots(1, 1, figsize=(5,5))
ax_det = axes
classifiers = {
    "Random Forest": grid_rf,
    "Random Forest with Over Sampling": grid_over_rf,
    "Random Forest with Class weights": grid_rfb,
        # Add more classifiers here if needed }
}
for name, clf in classifiers.items():
    clf.fit(Xpc_train, y_train)
    DetCurveDisplay.from_estimator(clf, Xpc_test, y_test, ax=ax_det, name=name)

ax_det.set_title("Detection Error Tradeoff (DET) curves")
plt.legend()
plt.show()

## **Findings**

#**Neural Network**



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install scikeras

In [ ]:
from scikeras.wrappers import KerasClassifier
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras import layers
from keras import models
from keras import utils
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.layers import Activation
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import Recall, FalseNegatives, AUC

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score, accuracy_score, roc_auc_score


## **Baseline Neural Network**

Define a baseline neural network with two layers, one with 100 neurons and a Relu activation function, and an output layer with a Sigmoid activation function.

The binary cross-entropy loss function and recall scoring method selected for as best suited for a binary classification with imbalanced data.

It isn't obvious which optimizer is best suited to this problem, so we will test three from the Keras libary on the base line model using default settings and compare recall scores:


*   Standard Gradient Descent
*   RMSprop - adaptive stochastic gradient descent
*   Adam - adaptive stochastic gradient descent



**SGD optimizer**

In [ ]:
#Baseline model testing SGD optimizer with default values
#Define metrics

#Define model
model = Sequential([
        Dense(100, activation = 'relu'),
        Dropout(0.3),
        Dense(1, activation = 'sigmoid')])

#Compile the model
model.compile(loss = 'binary_crossentropy',
              optimizer = 'sgd',
              metrics = ['recall'])

class_weight={0: 1, 1: 20}
model.summary()

In [ ]:
#Fit the model and view scores
#Batch size set by number of columns in Xpc_train
tf.random.set_seed(42)

# Convert y_test and Xpc_test to NumPy arrays
#y_train = y_train.values
#y_test = y_test.values

base = model.fit(Xpc_train, y_train,
                 epochs = 20,
                 batch_size = 22,
                 verbose = 0,
                 class_weight=class_weight,
                 validation_data=(Xpc_test, y_test))
#Score the model
base.history['recall'][-1]
model.summary()

In [ ]:
# Calculate AUC score
tf.random.set_seed(42)
y_pred_probs = model.predict(Xpc_test)
sgd_auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"AUC: {sgd_auc_score}")

In [ ]:
#Model validation and visualization
tf.random.set_seed(42)
sgd_score = model.evaluate(Xpc_test, y_test, verbose=1)
print('Test recall:', sgd_score[1])
print('Test loss:', sgd_score[0])

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['recall']), 'r', label='train_recall')
ax.plot(np.sqrt(base.history['val_recall']), 'b' ,label='val_recall')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Recall')
ax.legend()
ax.tick_params()

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['loss']), 'r', label='train')
ax.plot(np.sqrt(base.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Loss')
ax.legend()
ax.tick_params()

**RMSprop Optimizer**

In [ ]:
#Define a baseline neural network with two layers
#Loss functionand scoring selected for binary classification
#Testing RMSprop optimizer with default values

#Define model
model = Sequential([
        Dense(100, activation = 'relu'),
        Dropout(0.3),
        Dense(1, activation = 'sigmoid')])

#Compile the model
model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['recall'])
class_weight={0: 1, 1: 20}

In [ ]:
#Fit the model and view scores
#Batch size set by number of columns in Xpc_train
tf.random.set_seed(42)
base = model.fit(Xpc_train, y_train,
                 epochs = 20,
                 batch_size = 22,
                 verbose = 0,
                 class_weight=class_weight,
                 validation_data=(Xpc_test, y_test))
#Score the model
base.history['recall'][-1]

In [ ]:
# Calculate AUC score
tf.random.set_seed(42)
y_pred_probs = model.predict(Xpc_test)
rmsprop_auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"AUC: {rmsprop_auc_score}")

In [ ]:
#Model validation and visualization
tf.random.set_seed(42)
rmsprop_score = model.evaluate(Xpc_test, y_test, verbose=1)
print('Test recall:', rmsprop_score[1])
print('Test loss:', rmsprop_score[0])

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['recall']), 'r', label='train_recall')
ax.plot(np.sqrt(base.history['val_recall']), 'b' ,label='val_recall')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Recall')
ax.legend()
ax.tick_params()

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['loss']), 'r', label='train')
ax.plot(np.sqrt(base.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Loss')
ax.legend()
ax.tick_params()

**ADAM Optimizer**

In [ ]:
#Define a baseline neural network with two layers
#Loss functionand scoring selected for binary classification
#Testing RMSprop optimizer with default values

#Define model
model = Sequential([
        Dense(100, activation = 'relu'),
        Dropout(0.3),
        Dense(1, activation = 'sigmoid')])

#Compile the model
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['recall'])
class_weight={0: 1, 1: 20}

In [ ]:
#Fit the model and view scores
#Batch size set by number of columns in Xpc_train
tf.random.set_seed(42)
base = model.fit(Xpc_train, y_train,
                 epochs = 20,
                 batch_size = 22,
                 verbose = 0,
                 class_weight=class_weight,
                 validation_data=(Xpc_test, y_test))
#Score the model
base.history['recall'][-1]

In [ ]:
# Calculate AUC score
tf.random.set_seed(42)
y_pred_probs = model.predict(Xpc_test)
adam_auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"AUC: {adam_auc_score}")

In [ ]:
#Model validation and visualization
tf.random.set_seed(42)
adam_score = model.evaluate(Xpc_test, y_test, verbose=1)
print('Test recall:', adam_score[1])
print('Test loss:', adam_score[0])

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['recall']), 'r', label='train_recall')
ax.plot(np.sqrt(base.history['val_recall']), 'b' ,label='val_recall')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Recall')
ax.legend()
ax.tick_params()

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(base.history['loss']), 'r', label='train')
ax.plot(np.sqrt(base.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Loss')
ax.legend()
ax.tick_params()

In [ ]:
#Summary table
sgd = ['SGD', sgd_score[1], sgd_score[0], sgd_auc_score]
rmsprop = ['RMSprop', rmsprop_score[1], rmsprop_score[0], rmsprop_auc_score]
adam = ['Adam', adam_score[1], adam_score[0], adam_auc_score]

# Create the DataFrame
summary = pd.DataFrame(data=[sgd, rmsprop, adam],
                      columns=['Optimizer', 'Recall', 'Loss', 'ROC-AUC Score'])

print(summary)

## **Findings**

## **Hyperparameter Tuning**

We will tune some key parameters of the model using GridSearch for:

*   Initialization mode
*   Number of neurons
*   Batch size



In [ ]:
# Function to create model, required for KerasClassifier
def create_model(units=32):
    model = Sequential()
    model.add(Dense(units, input_dim=22, activation='relu'))
    model.add(Dropout(0.3)),
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='bce', optimizer='sgd', metrics=['recall'])
    return model

# Instantiate the KerasClassifier model
model = KerasClassifier(build_fn=create_model, verbose=1)
class_weight={0: 1, 1: 20}
#model.summary()


In [ ]:
# Define the parameter grid
param_grid = {'epochs': [10, 20,30],
              'model__units': [50, 100, 150]}

# Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, refit=True)
grid.fit(Xpc_train, y_train)

In [ ]:
print(grid.best_score_)
grid.best_params_

In [ ]:
from keras import initializers

In [ ]:
#Re-run the model with tuned parameters
model = Sequential()
model.add(Dense(100, input_dim=22, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='bce', optimizer='rmsprop', metrics=['recall'])
class_weight={0: 1, 1: 20}

In [ ]:
tf.random.set_seed(42)
# Convert y_train to a NumPy array to resolve the issue
y_train_np = y_train.values.ravel()

tuned = model.fit(Xpc_train, y_train_np,
                 epochs = 20,
                 batch_size = 22,
                 verbose = 1,
                 class_weight=class_weight,
                 validation_data=(Xpc_test, y_test))

In [ ]:
# Calculate AUC score
tf.random.set_seed(42)
y_pred_probs = model.predict(Xpc_test)
tuned_auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"AUC: {tuned_auc_score}")

In [ ]:
#Model validation and visualization
tf.random.set_seed(42)
tuned_score = model.evaluate(Xpc_test, y_test, verbose=1)
print('Test recall:', tuned_score[1])
print('Test loss:', tuned_score[0])

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(tuned.history['recall']), 'r', label='train_recall')
ax.plot(np.sqrt(tuned.history['val_recall']), 'b' ,label='val_recall')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Recall')
ax.legend()
ax.tick_params()

fig, ax = plt.subplots(1, 1, figsize=(5,5))
ax.plot(np.sqrt(tuned.history['loss']), 'r', label='train')
ax.plot(np.sqrt(tuned.history['val_loss']), 'b' ,label='val')
ax.set_xlabel(r'Epoch')
ax.set_ylabel(r'Loss')
ax.legend()
ax.tick_params()